# Cluster state virtual vector proj rep check

Created 10/04/2024

Objectives:
* Sweep over all cases, and perform optimisation to the left and right of the symmetry action.

# Package imports

In [1]:
import sys
sys.path.append('../')

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [3]:
from functools import reduce
from operator import mul

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries

from SPTOptimization.Optimizers.OneSiteSolver import OneSiteSolver

from SPTOptimization.utils import (
    to_npc_array,
    get_right_identity_environment,
    get_right_identity_environment_from_tp_tensor,
    multiply_transfer_matrices
)

from SPTOptimization.Optimizers.utils import (
    one_site_optimization_sweep_right
)

In [6]:
import re

# Load data

In [7]:
DATA_DIR = r"../data/transverse_cluster_200_site_dmrg"

In [8]:
loaded_data = list()

for local_file_name in os.listdir(DATA_DIR):
    f_name = r"{}/{}".format(DATA_DIR, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)
        loaded_data.append(data)

In [9]:
b_parameters = sorted(list(d['paramters']['B'] for d in loaded_data))

In [10]:
psi_dict = dict()

In [11]:
for b in b_parameters:
    psi = next(
        d['wavefunction']
        for d in loaded_data
        if d['paramters']['B'] == b
    )

    rounded_b = round(b, 1)
    psi_dict[rounded_b] = psi

# Definitons

Will need to update these based off previous work...

In [12]:
MAX_VIRTUAL_BOND_DIM = 8
MAX_INTERMEDIATE_VIRTUAL_BOND_DIM = 2*MAX_VIRTUAL_BOND_DIM
# MPO bond dim?
MAX_MPO_BOND_DIM = 50

SVD_CUTOFF = 1e-3

In [13]:
np_I = np.array([[1,0],[0,1]])
np_X = np.array([[0,1],[1,0]])
np_Y = np.array([[0,-1j],[1j,0]])
np_Z = np.array([[1,0],[0,-1]])

In [14]:
npc_I = to_npc_array(np_I)
npc_X = to_npc_array(np_X)
npc_Y = to_npc_array(np_Y)
npc_Z = to_npc_array(np_Z)

In [15]:
symmetry_actions = [
    [np_I, np_I],
    [np_I, np_X],
    [np_X, np_I],
    [np_X, np_X]
]

In [16]:
cases = [
    [
        SymmetryActionWithBoundaryUnitaries(
            psi,
            s*50,
            left_boundary_unitaries=[np_I,]*3,
            right_boundary_unitaries=[np_I,]*3
        )
        for s in symmetry_actions
    ]
    for psi in psi_dict.values()
]

In [17]:
for l in cases:
    for c in l:
        c.compute_svd_approximate_expectation()

In [18]:
left_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=1
)

In [19]:
right_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=-1
)

# Check phases

In [26]:
def normalise_scaled_unitary(matrix):
    m = matrix

    tr = npc.tensordot(
        m,
        m.conj(),
        [['vR', 'vR*'], ['vR*', 'vR']]
    )

    dim = m.to_ndarray().shape[0]

    scale = np.sqrt(tr/dim)

    out = m/scale

    return out

In [29]:
def multiply_right_environments(env1, env2):
    out = npc.tensordot(
        env1,
        env2,
        [['vR',], ['vR*']]
    )

    return out

In [30]:
def right_proj_rep_invariant_phase_from_environments_1(case1, case2):
    v1 = case1.right_projected_symmetry_state
    v1 = normalise_scaled_unitary(v1)

    #print(np.round(npc.tensordot(v1, v1.conj(), [['vR',], ['vR*']]).to_ndarray()), 2)

    v2 = case2.right_projected_symmetry_state

    v2 = normalise_scaled_unitary(v2)

    #print(np.round(npc.tensordot(v2, v2.conj(), [['vR',], ['vR*']]).to_ndarray()), 2)

    t = reduce(
        multiply_right_environments,
        [v1, v2, v1.conj(), v2.conj()]
    )

    exp = npc.trace(t)
    phase = exp/np.abs(exp)

    return phase

In [31]:
cases[0]

In [32]:
from itertools import combinations
from functools import reduce

In [35]:
phases = list()

for c in cases:
    case_pairs = list(combinations(c[1:], 2))
    
    l = [
        right_proj_rep_invariant_phase_from_environments_1(c1, c2)
        for c1, c2 in case_pairs
    ]

    phases.append(l)

In [36]:
phases

[[(-1+0j), (-1+0j), (-1+0j)],
 [(-1+0j), (-1+0j), (-1+0j)],
 [(-1+0j), (-1+0j), (-1+0j)],
 [(-1+0j), (-1+0j), (-1+0j)],
 [(-1+0j), (-1+0j), (-1+0j)],
 [(-1+0j), (-1+0j), (-1+0j)],
 [(-1+0j), (-1+0j), (-1+0j)],
 [(-0.9999999999999999+0j),
  (-0.9999999999999999+0j),
  (-0.9999999999999999+0j)],
 [(-1+0j), (-1+0j), (-1+0j)],
 [(-1+0j), (-1+0j), (-0.9999999999999999+0j)],
 [(1+0j), (-1+0j), (-1+0j)],
 [(1+0j), (1+0j), (1+0j)],
 [(1+0j), (1+0j), (1+0j)],
 [(1+0j), (1+0j), (1+0j)],
 [(1+0j), (1+0j), (1+0j)],
 [(1+0j), (1+0j), (1+0j)],
 [(1+0j), (1+0j), (1+0j)],
 [(1+0j), (1+0j), (1+0j)],
 [(1+0j), (1+0j), (1+0j)],
 [(1+0j), (1+0j), (1+0j)],
 [(0.9999999999999999+0j), (0.9999999999999999+0j), (0.9999999999999999+0j)]]